In [1]:
import googlemaps
from dotenv import load_dotenv
import os
import ast
import numpy as np
from geopy.distance import geodesic
import pandas as pd
import geopandas as gpd
import time
from pathlib import Path

PROJECT_ROOT = Path().absolute().parent
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
TEMP_DATA_DIR = DATA_DIR / 'temp'

#load in data 
sampled_clusters_gdf=gpd.read_file(PROCESSED_DATA_DIR/'sampled_clusters.gpkg')
sampled_clusters_gdf.station_name.unique()
sampled_clusters_gdf

#split this up in batches, dwanwana dokolo first then asia fm. 

dwanwana_dokolo_sampled_clusters_gdf=sampled_clusters_gdf.loc[sampled_clusters_gdf['station_name'].isin(['Dwanwana FM', 'Dokolo FM'])]
dwanwana_dokolo_sampled_clusters_gdf=dwanwana_dokolo_sampled_clusters_gdf.iloc[:5]
#['Aisa FM', 'Dwanwana FM', 'Dokolo FM']
# Load API key
load_dotenv()
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
gmaps = googlemaps.Client(key=api_key)

def get_points_on_circle(center_lat, center_lon, radius_meters, num_points=4):
    """Generate points on a circle around the center point."""
    radius_deg = radius_meters / 111000
    
    points = []
    for i in range(num_points):
        angle = 2 * np.pi * i / num_points
        lat = center_lat + radius_deg * np.cos(angle)
        lon = center_lon + radius_deg * np.sin(angle) / np.cos(np.radians(center_lat))
        points.append((lat, lon))
    
    return points

def find_nearest_road(center_lat, center_lon, gmaps_client, verbose=False):
    """
    Find the nearest road point to a given centroid.
    Stops searching as soon as a road is found in the first radius.
    """
    try:
        # Define search radii
        search_radii = [25, 50, 100, 150, 200]
        
        if verbose:
            print(f"\nOriginal centroid (lat, lon): {center_lat}, {center_lon}")
        
        closest_point = None
        min_distance = float('inf')
        
        # Search at increasing radii
        for radius in search_radii:
            if verbose:
                print(f"\nSearching at {radius} meters radius...")
            
            num_points = 4 if radius <= 50 else 8
            test_points = get_points_on_circle(center_lat, center_lon, radius_meters=radius, num_points=num_points)
            
            road_found = False  # Flag for finding any road
            
            for i, origin in enumerate(test_points):
                if verbose:
                    print(f"Trying direction search {i+1}/{num_points} at {radius}m radius")
                
                destination = (center_lat, center_lon)
                
                # Get directions
                directions_result = gmaps_client.directions(
                    origin=origin,
                    destination=destination,
                    mode="driving"
                )
                
                if directions_result:
                    steps = directions_result[0]['legs'][0]['steps']
                    last_step = steps[-1]
                    
                    road_lat = last_step['start_location']['lat']
                    road_lon = last_step['start_location']['lng']
                    
                    distance = geodesic(
                        (center_lat, center_lon), 
                        (road_lat, road_lon)
                    ).meters
                    
                    if verbose:
                        print(f"Found road point at distance: {distance:.1f}m")
                    
                    road_found = True
                    
                    if distance < min_distance:
                        min_distance = distance
                        closest_point = {
                            'lat': road_lat,
                            'lon': road_lon,
                            'distance': distance,
                            'found_at_radius': radius
                        }
            
            # If we found any road in this radius, stop searching
            if road_found and radius == 25:  # Only stop if we found a road in the first radius
                if verbose:
                    print(f"\nFound road within first radius ({radius}m), stopping search")
                break
        
        return closest_point
                
    except Exception as e:
        if verbose:
            print(f"Error: {e}")
        return None

In [5]:
def process_all_clusters(sampled_clusters_gdf):
    """
    Process all clusters to find nearest roads for their centroids.
    Returns a DataFrame with results.
    """
    # Initialize results list
    results = []
    
    # Total number of clusters for progress tracking
    total_clusters = len(sampled_clusters_gdf)
    print(f"Starting to process {total_clusters} clusters...")
    
    for idx, row in sampled_clusters_gdf.iterrows():
        print(f"\nProcessing {idx+1}/{total_clusters}: grid_id {row.grid_id}")
        
        try:
            # Get centroid coordinates
            coord_tuple = ast.literal_eval(row.centroid_lon_lat)
            center_lat, center_lon = coord_tuple[1], coord_tuple[0]
            
            # Find nearest road
            closest_point = find_nearest_road(center_lat, center_lon, gmaps, verbose=True)
            
            # Store results
            result = {
                'grid_id': row.grid_id,
                'station_name': row.station_name,
                'buffer_km':row.buffer_km,
                'est_population_2020':row.population_count,
                'centroid_lat': center_lat,
                'centroid_lon': center_lon,
                'centroid_maps_link': f"https://www.google.com/maps?q={center_lat},{center_lon}",
                'cluster_type': row.cluster_type,
                'nearest_road_lat': closest_point['lat'] if closest_point else None,
                'nearest_road_lon': closest_point['lon'] if closest_point else None,
                'distance_to_road': closest_point['distance'] if closest_point else None,
                'nearest_road_maps_link': (f"https://www.google.com/maps?q={closest_point['lat']},{closest_point['lon']}" 
                                         if closest_point else None),
                'found_at_radius': closest_point['found_at_radius'] if closest_point else None,
                'processing_success': True if closest_point else False,
                'geometry_grid_cell': row.geometry.wkt  
            }
            
            results.append(result)
            
            # Add a small delay to avoid hitting API limits
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Error processing grid_id {row.grid_id}: {e}")
            # Store error result
            result = {
                'grid_id': row.grid_id,
                'station_name': row.station_name,
                'buffer_km':row.buffer_km,
                'est_population_2020':row.population_count,
                'centroid_lat': center_lat,
                'centroid_lon': center_lon,
                'centroid_maps_link': f"https://www.google.com/maps?q={center_lat},{center_lon}",
                'cluster_type': row.cluster_type,
                'nearest_road_lat': None,
                'nearest_road_lon': None,
                'distance_to_road': None,
                'nearest_road_maps_link': None,
                'found_at_radius': None,
                'processing_success': False,
                'geometry_grid_cell': row.geometry.wkt  
            }
            results.append(result)
    
    # Convert results to DataFrame and return
    return pd.DataFrame(results)

# Usage:
results_df = process_all_clusters(dwanwana_dokolo_sampled_clusters_gdf)

Starting to process 5 clusters...

Processing 71/5: grid_id 8263

Original centroid (lat, lon): 1.8581265161490297, 33.056539360415755

Searching at 25 meters radius...
Trying direction search 1/4 at 25m radius
Found road point at distance: 51.8m
Trying direction search 2/4 at 25m radius
Found road point at distance: 25.1m
Trying direction search 3/4 at 25m radius
Found road point at distance: 33.9m
Trying direction search 4/4 at 25m radius
Found road point at distance: 25.1m

Found road within first radius (25m), stopping search

Processing 72/5: grid_id 9989

Original centroid (lat, lon): 2.093347890770116, 33.14647425120548

Searching at 25 meters radius...
Trying direction search 1/4 at 25m radius
Found road point at distance: 137.9m
Trying direction search 2/4 at 25m radius
Found road point at distance: 137.9m
Trying direction search 3/4 at 25m radius
Found road point at distance: 137.9m
Trying direction search 4/4 at 25m radius
Found road point at distance: 137.9m

Found road wit

In [7]:
results_df

,grid_id,station_name,buffer_km,est_population_2020,centroid_lat,centroid_lon,centroid_maps_link,cluster_type,nearest_road_lat,nearest_road_lon,distance_to_road,nearest_road_maps_link,found_at_radius,processing_success,geometry_grid_cell
0,8263,Dwanwana FM,25.0,179.262650,1.858127,33.056539,https://www.google.com/maps?q=1.85812651614902...,replacement,1.858005,33.056349,25.063619,"https://www.google.com/maps?q=1.858005,33.0563492",25,True,"POLYGON ((33.06103492495046 1.853602761066021,..."
1,9989,Dwanwana FM,25.0,381.704590,2.093348,33.146474,https://www.google.com/maps?q=2.09334789077011...,replacement,2.094389,33.147156,137.861247,"https://www.google.com/maps?q=2.0943889,33.147...",25,True,"POLYGON ((33.15097015834338 2.088823883597678,..."
2,13719,Dwanwana FM,25.0,341.559753,2.002846,33.344293,https://www.google.com/maps?q=2.00284642067057...,replacement,2.005120,33.343419,269.533294,"https://www.google.com/maps?q=2.0051201,33.343...",25,True,POLYGON ((33.34878766894459 1.9983219495517408...
3,12682,Dwanwana FM,25.0,324.227936,1.849056,33.290314,https://www.google.com/maps?q=1.84905632797708...,replacement,1.849347,33.289630,82.581111,"https://www.google.com/maps?q=1.8493468,33.289...",25,True,POLYGON ((33.294809077609216 1.844532035315588...
4,9098,Dwanwana FM,25.0,290.368835,1.722416,33.101489,https://www.google.com/maps?q=1.72241633301866...,main,1.722582,33.101458,18.653578,"https://www.google.com/maps?q=1.7225822,33.101...",25,True,"POLYGON ((33.1059842060682 1.7178924806024494,..."


In [ ]:
dwanwana_dokolo_sampled_clusters_gdf

In [8]:
def create_enumeration_area_map(row):
    """
    Create a map for a single enumeration area showing:
    1. The nearest road point
    2. The grid cell geometry
    3. The centroid
    
    Parameters:
    row: A single row from the results DataFrame
    """
    import folium
    from shapely import wkt
    
    # Convert WKT string back to geometry
    grid_geometry = wkt.loads(row.geometry)
    
    # Create a map centered on the nearest road point (if it exists) or centroid
    if row.nearest_road_lat and row.nearest_road_lon:
        center_lat, center_lon = row.nearest_road_lat, row.nearest_road_lon
    else:
        center_lat, center_lon = row.centroid_lat, row.centroid_lon
        
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=17,
        tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr='Google Maps'
    )
    
    # Add the grid cell geometry
    folium.GeoJson(
        grid_geometry.__geo_interface__,
        style_function=lambda x: {
            'fillColor': '#ffaf00',
            'color': '#ff0000',
            'weight': 2,
            'fillOpacity': 0.1
        },
        popup=f"Grid ID: {row.grid_id}"
    ).add_to(m)
    
    # Add the centroid point
    folium.CircleMarker(
        location=[row.centroid_lat, row.centroid_lon],
        radius=6,
        color='blue',
        fill=True,
        popup='Centroid'
    ).add_to(m)
    
    # Add the nearest road point if it exists
    if row.nearest_road_lat and row.nearest_road_lon:
        folium.CircleMarker(
            location=[row.nearest_road_lat, row.nearest_road_lon],
            radius=6,
            color='red',
            fill=True,
            popup=f'Nearest Road Point (Distance: {row.distance_to_road:.1f}m)'
        ).add_to(m)
        
        # Add a line connecting centroid to nearest road point
        folium.PolyLine(
            locations=[
                [row.centroid_lat, row.centroid_lon],
                [row.nearest_road_lat, row.nearest_road_lon]
            ],
            weight=2,
            color='red',
            opacity=0.8
        ).add_to(m)
    
    # Add a legend
    legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; 
         background-color: white; padding: 10px; border: 2px solid grey; border-radius: 5px;">
        <p><i style="background: red; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Nearest Road</p>
        <p><i style="background: blue; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Centroid</p>
        <p><i style="border: 2px solid red; width: 10px; height: 10px; display: inline-block;"></i> Grid Boundary</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

# Create maps for all enumeration areas
for idx, row in results_df.iterrows():
    m = create_enumeration_area_map(row)
    m.save(f'enumeration_area_map_{row.grid_id}.html')

# Or test with just one area
test_map = create_enumeration_area_map(results_df.iloc[0])
test_map.save('test_enumeration_area_map.html')

AttributeError: 'Series' object has no attribute 'geometry'